In [6]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.utils import to_categorical
from keras.datasets import cifar10
from keras.applications.xception import Xception
from keras.layers import Dense, Flatten, Dropout
from keras.models import Sequential

In [7]:
# Load CIFAR-10 dataset and preprocess the data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Split the training set into training and validation sets
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)


In [8]:
model_xception = Xception(weights='imagenet', include_top=False, input_shape=(71, 71, 3))


In [9]:
inp = keras.Input(shape=(32, 32 ,3))
inp_resized = keras.layers.Lambda(lambda X: tf.image.resize(X, (71, 71)))(inp)

In [10]:
X = model_xception(inp_resized, training=False)
X = Flatten()(X)
X = Dense(500, activation='relu')(X)
X - Dropout(0.3)(X)
outputs = Dense(10, activation='softmax')(X)
model = keras.Model(inp, outputs)
model_xception.trainable = False
model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model using the validation set for monitoring
batch_size = 32
epochs = 20
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val), verbose=1)

Epoch 1/20
1407/1407 [==============================] - 36s 17ms/step - loss: 0.1772 - accuracy: 0.6762 - val_loss: 0.1542 - val_accuracy: 0.6928
Epoch 2/20
1407/1407 [==============================] - 20s 15ms/step - loss: 0.1300 - accuracy: 0.7424 - val_loss: 0.1358 - val_accuracy: 0.7284
Epoch 3/20
1407/1407 [==============================] - 20s 15ms/step - loss: 0.1122 - accuracy: 0.7782 - val_loss: 0.1325 - val_accuracy: 0.7362
Epoch 4/20
1407/1407 [==============================] - 20s 14ms/step - loss: 0.0971 - accuracy: 0.8138 - val_loss: 0.1323 - val_accuracy: 0.7348
Epoch 5/20
1407/1407 [==============================] - 21s 15ms/step - loss: 0.0833 - accuracy: 0.8453 - val_loss: 0.1398 - val_accuracy: 0.7300
Epoch 6/20
1407/1407 [==============================] - 21s 15ms/step - loss: 0.0704 - accuracy: 0.8764 - val_loss: 0.1410 - val_accuracy: 0.7434
Epoch 7/20
1407/1407 [==============================] - 21s 15ms/step - loss: 0.0582 - accuracy: 0.9051 - val_loss: 0.1496 -

In [11]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test, batch_size=batch_size)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

313/313 [==============================] - 4s 13ms/step - loss: 0.2946 - accuracy: 0.7329
Test Loss: 0.294609010219574
Test Accuracy: 0.7329000234603882
